# Libraries + Torch check

In [33]:
import pandas as pd
import requests
import json
from pathlib import Path
import re
from datetime import date
import time
import os
from fastai.tabular import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import sklearn.metrics
from sklearn.model_selection import train_test_split
import math

In [34]:
import torch

In [35]:
torch.cuda.device(0)

In [36]:
torch.cuda.get_device_name(0)

'GeForce RTX 2080'

# Data Retrieval

In [37]:
cleaned_item_names = [name[:-10] for name in list(os.walk(r'data/wiki_prices'))[0][2]]

In [38]:
cleaned_item_names[0:5]

['Potato%20seed', 'Tin%20ore', 'Gout%20tuber', 'Bronze%20bolts', 'Gold%20seal']

In [39]:
pd.Series(cleaned_item_names).to_csv('./data/cleaned_item_names', index=False, header=False)

In [40]:
len(cleaned_item_names)

3147

In [41]:
# cleaned_item_names

In [44]:
def get_item_df(item_name, sleppy_time=1):
    base_url = r"https://oldschool.runescape.wiki/api.php?action=query&prop=revisions&rvprop=content&format=json&titles=Module%3AExchange%2F"
    postfix_url = r"%2FData"
    full_url = f'{base_url}{item_name}{postfix_url}'
    resp = requests.get(full_url)
    jsond = resp.json()
    try:
        regex_text = jsond['query']['pages'][next(iter(jsond['query']['pages'].keys()))]['revisions'][0]['*']
        #get all of the ones that have volume as well as price
        m = re.findall(r'[\d]+:[\d]+:[\d]+', regex_text)
        item_df = pd.DataFrame()
        item_df['dt'] = [int(string.split(':')[0]) for string in m]
        item_df[f'{item_name} price'] = [int(string.split(':')[1]) for string in m]
        item_df[f'{item_name} volume'] = [int(string.split(':')[2]) for string in m]
        item_df['date_obj'] = item_df['dt'].apply(lambda x: date.fromtimestamp(x))
        #take the ones that just have price so we can get the most recent price as well
        n = re.findall(r'[\d]+:[\d]+', regex_text)
        last_time = date.fromtimestamp(int(n[-1].split(':')[0]))
        last_price = int(n[-1].split(':')[1])
        item_df.loc[-1, 'date_obj'] = last_time
        item_df.loc[-1, f'{item_name} price'] = last_price
        item_df = item_df.fillna(method='ffill')
        item_df.set_index('date_obj', inplace=True)
        item_df = item_df.drop('dt', axis=1)
        return item_df
    except:
        pass



In [45]:
def retrieve_dfs(xslice = True):
    if xslice:
        item_dfs = [get_item_df(item_name,1) for item_name in cleaned_item_names[40:45]]
    else:
        item_dfs = [get_item_df(item_name,1) for item_name in cleaned_item_names]
    return item_dfs

In [46]:
item_dfs = retrieve_dfs(False)

In [47]:
def save_dfs(item_dfs):
    for df in item_dfs:
        try:
            df.to_csv(f'./data/wiki_prices/{df.columns[0]}.csv')
        except:
            pass

In [49]:
save_dfs(item_dfs)

In [50]:
def load_dfs(drop_blank=True):
    item_dfs = []
    for name in list(os.walk('./data/wiki_prices'))[0][2]:
        df = pd.read_csv(f'./data/wiki_prices/{name}')
        df = df.set_index(df.columns[0])
#         if drop_blank:
#             if df.empty:
#                 item_dfs.append(df)
#         else:
#             item_dfs.append(df)
        item_dfs.append(df)
    return item_dfs

In [16]:
item_dfs = load_dfs()

In [51]:
len(item_dfs)

3147

In [52]:
type(item_dfs)

list

In [53]:
cleaned_item_names[0]

'Potato%20seed'

In [54]:
for item_df in item_dfs:
    try:
        col = item_df.columns[0]
    except:
        invalid_dfs += 1 
        print(item_df)

In [55]:
retrieved_names = [item_df.columns[0][:-6] for item_df in item_dfs]

In [56]:
len(retrieved_names)

3147

In [57]:
len(item_dfs)

3147

In [58]:
df = pd.DataFrame({'name': retrieved_names, 'price_df': item_dfs})

# Feature creation

In [59]:
#item_df_training = tl_df['price_df'][300]

In [60]:
days_back = 90

rolling_averages = [3,7,14,21,30,60,90]

days_ahead_to_predict = 4

In [61]:
def create_predictive_features(price_df):
    item_df_training = price_df
    item_name = item_df_training.columns[0]
    for i in range(days_back):
        item_df_training[f'{item_name} {i} days ago'] = item_df_training[item_name].shift(i)
    for i in range(days_back):
        item_df_training[f'{item_name} {i} delta days ago'] = item_df_training[item_name].shift(i) - item_df_training[item_name]
    for rolling_average in rolling_averages:
        item_df_training[f'{item_name} {rolling_average} day rolling average'] = item_df_training[item_name].rolling(rolling_average).mean()
    for rolling_average in rolling_averages:
        item_df_training[f'{item_name} delta {rolling_average} day rolling average'] = item_df_training[item_name] - item_df_training[f'{item_name} {rolling_average} day rolling average']
    item_df_training['Y'] = item_df_training[item_name].shift(-1*days_ahead_to_predict)
    return item_df_training

        

In [62]:
#price_df = tl_df['price_df'][300]

In [63]:
def trim_df(item_df_training):
    trimmed_df = item_df_training.dropna(how='any')
    return trimmed_df
    

# SGD LR

In [64]:
def create_model(tl_df_row):
    trimmed_df = tl_df_row['trimmed_df']
    try:
        X = trimmed_df.drop(['Y'], axis=1)
        y = trimmed_df['Y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        mean_squared_error = sklearn.metrics.mean_squared_error(y_test, y_pred)
        r2_score = sklearn.metrics.r2_score(y_test, y_pred)
        return (regr, y_pred, y_test, mean_squared_error, r2_score)
    except:
        return (0, 0, 0, 0, 0)

In [65]:
#plt.scatter(y_test, y_pred)

# "Backtest"

In [66]:
starting_cash = 10**7

In [67]:
slippage = 0
thresh_buy = 0.00
thresh_sell = 0
max_volume_buy = 0.01

In [68]:
def should_buy(row):
    if row['perc predicted increase'] > thresh_buy:
        return True

def should_sell(row):name
Icy%20basalt
Black%20shield%20(h1)
Saradomin%20bracers
Merfolk%20trident
Mithril%20kiteshield

    if row['perc predicted increase'] < thresh_sell:
        return True
    
def buy_price(row):
    return row['current_price']

def sell_price(row):
    return row['current_price']

def max_buy(row):
    d_stock = max(
        min(
        (row['starting_cash']//buy_price(row)),
        math.floor(row['volume']*max_volume_buy)
        ),
        0
    )
    d_cash = -1 * d_stock * buy_price(row)
    return (d_stock, d_cash)

def max_sell(row):
    d_stock = -1 * max(
        0,
        min(row['starting_stock'],
        math.floor(row['volume'] * max_volume_buy)
        )
    )
    d_cash = d_stock *-1 * sell_price(row)
    return (d_stock, d_cash)

In [69]:
def get_ending_cash(record_df):
    last_row = record_df.iloc[-1,:]
    cash = last_row['ending_cash']
    cash += sell_price(last_row) * last_row['ending_stock']
    return cash

In [70]:
def create_backtest_df(tl_df_row):
    try:
        y_pred = tl_df_row['y_pred']
        y_test = tl_df_row['y_test']
        item_df_training = tl_df_row['item_df_training']
        item_name = tl_df_row['name']

        bt_df = pd.concat([pd.Series(y_pred, index= y_test.index, name='Prediction'), y_test.rename(index='Y')], axis=1)
        bt_df['Y + 1'] = bt_df['Y'].shift(-1)
        bt_df['Y - 1'] = bt_df['Y'].shift(1)


        bt_df = bt_df.fillna(method='ffill').fillna(method='bfill')

        volume_column = f'{item_name} volume'
        volume = item_df_training[volume_column]

        bt_df = bt_df.join(item_df_training[volume_column].rename(index='volume'))
        bt_df = bt_df.join(item_df_training[f'{item_name} price'].rename(index='current_price'))
        bt_df['perc predicted increase'] = (bt_df['Prediction'] - bt_df['current_price'])/bt_df['current_price']

        bt_df['starting_stock'] = 0
        bt_df['starting_cash'] = 0
        bt_df['ending_stock'] = 0
        bt_df['ending_cash'] = 0

        bt_df.loc[bt_df.index[0], 'starting_cash'] = starting_cash
        bt_df.loc[bt_df.index[0], 'ending_cash'] = starting_cash

        bt_df['action'] = ''

        copy_df = bt_df.copy()
        record_df = bt_df.copy()

        first_row = True
        for i, row in copy_df.iterrows():
            current_row = row
            if first_row:
                prev_row = current_row
                first_row = False
                continue
            current_row['starting_cash'] = prev_row['ending_cash']
            current_row['starting_stock'] = prev_row['ending_stock']
            cash = current_row['starting_cash']
            stock = current_row['starting_stock']
            if should_buy(current_row):
                d_stock, d_cash = max_buy(current_row)
                cash += d_cash
                stock += d_stock
                current_row['action'] = f'Buying + {d_stock} stock, - {d_cash} cash'
            elif should_sell(row):
                d_stock, d_cash = max_sell(current_row)
                cash += d_cash
                stock += d_stock
                current_row['action'] = f'Selling + {d_stock} stock, + {d_cash} cash'
            else:
                current_row['action'] = 'hold'
            current_row['ending_cash'] = cash
            current_row['ending_stock'] = stock
            record_df.loc[i, :] = current_row
            prev_row = current_row
        roi = get_ending_cash(record_df)/starting_cash
        last_perc_prediction = bt_df
        return (roi, record_df)
    except:
        return (0, 0)


# Predict for right now

In [71]:
def make_predictions(tl_df_row):
    try:
        item_name = tl_df_row['name']
        regr = tl_df_row['regr']
        item_df_training = tl_df_row['item_df_training']
        current_price = item_df_training.loc[item_df_training.index[-1], f'{item_name} price']
        pred_price = regr.predict(np.array(item_df_training.drop('Y', axis=1).loc[item_df_training.index[-1],:]).reshape(1, -1))[0]
        perc_pred_increase = (pred_price - current_price)/(current_price)
        last_day = item_df_training.index[-1]
        return (perc_pred_increase, last_day)
    except:
        return(0, 0)

# Filter and find trades

In [72]:
r2_score_thresh = .85
roi_thresh = 1.05
perc_pred_increase_thresh = 0.03

In [73]:
batch_size = 100


In [74]:
df_list = []

In [77]:
bought = pd.read_csv('data/bought')


In [79]:
bought['name'].values

array(['Icy%20basalt', 'Black%20shield%20(h1)', 'Saradomin%20bracers', 'Merfolk%20trident', 'Mithril%20kiteshield'],
      dtype=object)

In [80]:
for i in range(0,df.shape[0],batch_size):
    tl_df = df.iloc[i:i+100,:]
    
    tl_df['item_df_training'] = tl_df['price_df'].apply(create_predictive_features)

    tl_df['trimmed_df'] = tl_df['item_df_training'].apply(trim_df)

    tl_df['regr'], tl_df['y_pred'], tl_df['y_test'], tl_df['mean_squared_error'], tl_df['r2_score'] = zip(*tl_df.apply(create_model, axis=1))


    tl_df['roi'], tl_df['record_df'] = zip(*tl_df.apply(create_backtest_df, axis=1))


    tl_df.sort_values(by='roi', ascending = False)

    tl_df['perc_pred_increase'], tl_df['last_day'] = zip(*tl_df.apply(make_predictions, axis=1))


    filter_tl_df = tl_df.copy()

    filter_tl_df = filter_tl_df.dropna(how='any')

    filter_tl_df =filter_tl_df.loc[filter_tl_df['r2_score'].apply(lambda x: type(x)) != list]

    filter_tl_df = filter_tl_df.loc[filter_tl_df['r2_score'] > r2_score_thresh]

    print(filter_tl_df.shape)

    filter_tl_df = filter_tl_df.loc[filter_tl_df['roi'] > roi_thresh]

    print(filter_tl_df.shape)

    filter_tl_df = filter_tl_df.loc[
            (filter_tl_df['perc_pred_increase'] > perc_pred_increase_thresh) \
        | (filter_tl_df['name'].isin(bought['name']))
                                   ]

    print(filter_tl_df.shape)

    print(filter_tl_df)
    df_list.append(filter_tl_df)

/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(22, 13)
(2, 13)
(0, 13)
Empty DataFrame
Columns: [name, price_df, item_df_training, trimmed_df, regr, y_pred, y_test, mean_squared_error, r2_score, roi, record_df, perc_pred_increase, last_day]
Index: []


/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(24, 13)
(1, 13)
(1, 13)
                           name  \
162  Black%20wizard%20hat%20(g)   

                                              price_df  \
162              Black%20wizard%20hat%20(g) price  ...   

                                      item_df_training  \
162              Black%20wizard%20hat%20(g) price  ...   

                                            trimmed_df                regr  \
162              Black%20wizard%20hat%20(g) price  ...  LinearRegression()   

                                                y_pred  \
162  [377761.0829621381, 385669.7704621381, 369986....   

                                                y_test  mean_squared_error  \
162  date_obj
2020-04-06    382445.0
2020-04-07    ...        1.035652e+08   

     r2_score       roi                                          record_df  \
162  0.963592  1.154134                 Prediction         Y     Y + 1 ...   

     perc_pred_increase    last_day  
162            0.076677  2020-08-01  


/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

...

/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(21, 13)
(4, 13)
(2, 13)
                  name                                           price_df  \
3020   Mole%20slippers              Mole%20slippers price  Mole%20slip...   
3066  Black%20cavalier              Black%20cavalier price  Black%20ca...   

                                       item_df_training  \
3020              Mole%20slippers price  Mole%20slip...   
3066              Black%20cavalier price  Black%20ca...   

                                             trimmed_df                regr  \
3020              Mole%20slippers price  Mole%20slip...  LinearRegression()   
3066              Black%20cavalier price  Black%20ca...  LinearRegression()   

                                                 y_pred  \
3020  [105275.3906123584, 106336.1581904834, 105498....   
3066  [14002.993931361609, 13695.531529017859, 13745...   

                                                 y_test  mean_squared_error  \
3020  date_obj
2020-05-11    106416.0
2020-05-12    ...        2.12720

/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(17, 13)
(3, 13)
(0, 13)
Empty DataFrame
Columns: [name, price_df, item_df_training, trimmed_df, regr, y_pred, y_test, mean_squared_error, r2_score, roi, record_df, perc_pred_increase, last_day]
Index: []


/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/adamh/PycharmProjects/stonks/model/venv/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [81]:
filter_tl_df

,name,price_df,item_df_training,trimmed_df,regr,y_pred,y_test,mean_squared_error,r2_score,roi,record_df,perc_pred_increase,last_day


In [82]:
df_list

[Empty DataFrame
 Columns: [name, price_df, item_df_training, trimmed_df, regr, y_pred, y_test, mean_squared_error, r2_score, roi, record_df, perc_pred_increase, last_day]
 Index: [],
                            name  \
 162  Black%20wizard%20hat%20(g)   
 
                                               price_df  \
 162              Black%20wizard%20hat%20(g) price  ...   
 
                                       item_df_training  \
 162              Black%20wizard%20hat%20(g) price  ...   
 
                                             trimmed_df                regr  \
 162              Black%20wizard%20hat%20(g) price  ...  LinearRegression()   
 
                                                 y_pred  \
 162  [377761.0829621381, 385669.7704621381, 369986....   
 
                                                 y_test  mean_squared_error  \
 162  date_obj
 2020-04-06    382445.0
 2020-04-07    ...        1.035652e+08   
 
      r2_score       roi                                    

In [83]:
trading_df = pd.concat(df_list, axis=0).sort_values(by='perc_pred_increase', ascending=False)
trading_df.columns

Index(['name', 'price_df', 'item_df_training', 'trimmed_df', 'regr', 'y_pred',
       'y_test', 'mean_squared_error', 'r2_score', 'roi', 'record_df',
       'perc_pred_increase', 'last_day'],
      dtype='object')

In [84]:
trading_df.loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day']]

,name,mean_squared_error,r2_score,roi,perc_pred_increase,last_day
403,Black%20shield%20(h1),1.470758e+05,0.934341,1.061357,0.084099,2020-08-01
2469,Bottomless%20compost%20bucket,1.518091e+09,0.933250,1.240082,0.078670,2020-08-01
162,Black%20wizard%20hat%20(g),1.035652e+08,0.963592,1.154134,0.076677,2020-08-01
1159,Icy%20basalt,5.062121e+03,0.859634,1.085973,0.057565,2020-08-01
2225,Raw%20shark,2.433366e+03,0.935607,1.127036,0.055545,2020-08-01
1010,Maple%20longbow%20(u),7.470698e+00,0.957066,2.403655,0.048451,2020-08-01
1338,Merfolk%20trident,1.106398e+09,0.906930,1.076371,0.047365,2020-08-01
813,Black%20dragon%20mask,5.646106e+05,0.912560,1.055636,0.044488,2020-08-01
2826,Black%20d'hide%20chaps%20(g),1.561122e+09,0.895814,1.446377,0.039559,2020-08-01
2098,Lantadyme%20seed,2.653177e+04,0.907678,1.073880,0.035541,2020-08-01


In [121]:
trading_df.loc[trading_df['name'].isin(bought)].loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day']]

,name,mean_squared_error,r2_score,roi,perc_pred_increase,last_day
403,Black%20shield%20(h1),1.470758e+05,0.934341,1.061357,0.084099,2020-08-01
2469,Bottomless%20compost%20bucket,1.518091e+09,0.933250,1.240082,0.078670,2020-08-01
1159,Icy%20basalt,5.062121e+03,0.859634,1.085973,0.057565,2020-08-01
2225,Raw%20shark,2.433366e+03,0.935607,1.127036,0.055545,2020-08-01
1338,Merfolk%20trident,1.106398e+09,0.906930,1.076371,0.047365,2020-08-01
1564,Saradomin%20bracers,5.018610e+06,0.879335,1.088994,0.009442,2020-08-01


In [126]:
bought

0                        Icy%20basalt
1               Black%20shield%20(h1)
2                 Saradomin%20bracers
3                   Merfolk%20trident
2225                      Raw%20shark
2469    Bottomless%20compost%20bucket
Name: name, dtype: object

In [131]:
bought = bought.drop([2,3])

In [108]:
bought = pd.Series(['Icy%20basalt', 'Black%20shield%20(h1)', 'Saradomin%20bracers', 'Merfolk%20trident'], name='name')

In [109]:
bought

0             Icy%20basalt
1    Black%20shield%20(h1)
2      Saradomin%20bracers
3        Merfolk%20trident
Name: name, dtype: object

In [116]:
currently_buying = trading_df['name'][1:2]
currently_buying


2469    Bottomless%20compost%20bucket
Name: name, dtype: object

In [117]:
bought = bought.append(currently_buying)

In [132]:
#bought.merge(currently_buying)
bought.to_csv('data/bought', index=False)

In [133]:
pd.read_csv('data/bought')

,name
0,Icy%20basalt
1,Black%20shield%20(h1)
2,Raw%20shark
3,Bottomless%20compost%20bucket


In [ ]:
trading_

In [88]:
trading_df.loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day']].to_csv('data/trading_8.2.csv')

# fastai tabular data

In [ ]:
procs = [Normalize]

In [ ]:
valid_idx = range(int(len(df)*.8), len(df)-60)

In [ ]:
dep_var = 'Y'

In [ ]:
path = Path(f'./data/wiki_urls/{item_name}')

In [ ]:
trimmed_df['date_obj'] = trimmed_df.index

In [ ]:
trimmed_df['date_obj'] = trimmed_df['date_obj'].apply(lambda x: pd.to_datetime(x))

In [ ]:
time_trimmed_df = trimmed_df.loc[trimmed_df['date_obj'] > pd.to_datetime('2018-05-01')]

In [ ]:
time_trimmed_df[item_name].plot()

In [ ]:
time_trimmed_df = time_trimmed_df.drop(columns=['date_obj'], axis=1)

In [ ]:
trimmed_df = time_trimmed_df

In [ ]:
df = trimmed_df

In [ ]:
df

In [ ]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=[])

In [ ]:
learn = tabular_learner(data, layers=[500,100], emb_szs={'native-country': 10}, metrics=mean_squared_error)

In [ ]:
data.show_batch()

In [ ]:
learn.save('stage-0')

In [ ]:
learn.lr_find(end_lr=1e3)

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2)

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2)

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.save('stage-1')

In [ ]:
learn = learn.load('stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-5)

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.save('stage-2')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-6)

In [ ]:
learn.save('stage-3')


In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-4)

In [ ]:
learn.save('stage-4')

In [ ]:
learn.lr_find(end_lr=1e2)
learn.recorder.plot()

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.fit_one_cycle(10, 1e-4)

In [ ]:
learn = learn.load('stage-4')

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

# Plot results

In [ ]:
int(learn.predict(df.iloc[0])[1])

In [ ]:
learn.predict(df.iloc[0])

In [ ]:
plot_x = [float(learn.predict(df.iloc[i])[1]) for i in range(len(df))]

In [ ]:
plot_y = df['Y']

In [ ]:
plot_y

In [ ]:
plot_x

In [ ]:
plot_df = pd.DataFrame(plot_y)

In [ ]:
plot_df['x'] = plot_x

In [ ]:
plot_df.head()

In [ ]:
plot_df.plot()

In [ ]:
plt.plot(plot_x)

In [ ]:
plt.plot(plot_y)

In [ ]:
plt.plot(range(len(plot_x)),np.array([plot_x,plot_y]).T)

In [ ]:
len(plot_x)

In [ ]:
plt.plot(range(len(plot_x[len(plot_x)-60:])), np.array([plot_x[len(plot_x)-60:], plot_y[len(plot_y)-60:]]).T)
plt.legend()

In [ ]:
plt.plot(plot_x[len(plot_x)-60:])

In [ ]:
len(plot_x)

In [ ]:
plt.plot(plot_y[len(plot_y)-60:])

In [ ]:
x_series = pd.Series(plot_x, name='Predict')

In [ ]:
type(x_series)

In [ ]:
type(plot_y)

In [ ]:
y_series = plot_y.reindex(x_series.index)

In [ ]:
y_series

In [ ]:
x_series

In [ ]:
pd.concat([y_series, x_series],  axis=1).iloc[1705-60:].plot()

In [ ]:
learn.drop

In [ ]:
learn.save('stage-final')
learn.export()

# Load in the full data (recent rows will have been dropped because Ys were N/A)

In [ ]:
prediction_df = item_df_training

In [ ]:
prediction_df['constant'] = time_trimmed_df['Y'].mean() *scaling_factor

In [ ]:
prediction_df.columns

In [ ]:
prediction_df = prediction_df.fillna(method='ffill').fillna(method='bfill').fillna(0)

In [ ]:
predict_data = TabularDataBunch.from_df(path, prediction_df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=[])

In [ ]:
predict_learn = tabular_learner(data, layers=[500,100], emb_szs={'native-country': 10}, metrics=mean_squared_error)

In [ ]:
predict_learn =predict_learn.load('stage-final')

In [ ]:
predict_learn.show_results(ds_type=DatasetType.Train)

In [ ]:
predict_learn.predict(prediction_df.iloc[0])

In [ ]:
predict_learn.predict(prediction_df.iloc[100])

In [ ]:
prediction_df

In [ ]:
predictions = [float(predict_learn.predict(prediction_df.iloc[i])[1]) for i in range(len(prediction_df))]


In [ ]:
predictions

In [ ]:
prediction_df.tail()

In [ ]:
prediction_df['predictions'] = predictions

In [ ]:
prediction_df['predictions'].tail(30)

In [ ]:
prediction_df

#### 